# KISS Agent Framework - Interactive Tutorial

**Version:** 0.1.7

This notebook provides an interactive tutorial for the KISS (Keep It Simple, Stupid) Agent Framework.

KISS is a lightweight agent framework that implements a ReAct (Reasoning and Acting) loop for LLM agents.

## Prerequisites

Before running this notebook, ensure you have:
1. Installed the KISS framework: `uv sync --group dev`
2. Set up your API keys as environment variables

**Note:** Some cells require API keys to run (GEMINI_API_KEY, OPENAI_API_KEY, ANTHROPIC_API_KEY, etc.)

## 1. Setup and Environment Check

First, let's check that the KISS framework is properly installed and verify API key availability.

In [1]:
# Check KISS installation and version
import os
import sys
from IPython.display import display, Markdown

# Add src to path if running from the repo root
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from kiss import __version__

# Check available API keys
api_keys = {
    'GEMINI_API_KEY': os.environ.get('GEMINI_API_KEY'),
    'OPENAI_API_KEY': os.environ.get('OPENAI_API_KEY'),
    'ANTHROPIC_API_KEY': os.environ.get('ANTHROPIC_API_KEY'),
    'TOGETHER_API_KEY': os.environ.get('TOGETHER_API_KEY'),
    'OPENROUTER_API_KEY': os.environ.get('OPENROUTER_API_KEY'),
}

# Build status table
status_rows = "\n".join([
    f"| {key} | {'✓ Set' if value else '✗ Not set'} |"
    for key, value in api_keys.items()
])

display(Markdown(f"""
## KISS Framework Status

**Version:** `{__version__}`

### API Key Status

| Key | Status |
|-----|--------|
{status_rows}
"""))


## KISS Framework Status

**Version:** `0.1.9`

### API Key Status

| Key | Status |
|-----|--------|
| GEMINI_API_KEY | ✓ Set |
| OPENAI_API_KEY | ✓ Set |
| ANTHROPIC_API_KEY | ✓ Set |
| TOGETHER_API_KEY | ✓ Set |
| OPENROUTER_API_KEY | ✓ Set |


## 2. Your First Agent in 30 Seconds

Let's create a simple math agent that can evaluate expressions using tools.

KISS uses **native function calling** from the LLM providers. Your Python functions become tools automatically. Type hints become schemas. Docstrings become descriptions.

In [2]:
from IPython.display import display, Markdown
from kiss.core.kiss_agent import KISSAgent

def calculate(expression: str) -> str:
    """Evaluate a math expression."""
    return str(eval(expression))

agent = KISSAgent(name="Math Buddy")
result = agent.run(
    model_name="gemini-2.5-flash",
    prompt_template="Calculate: {question}",
    arguments={"question": "What is 15% of 847?"},
    tools=[calculate]
)
display(Markdown(f"**Result:** {result}"))


## role="user" #
Calculate: What is 15% of 847?
Asking gemini-2.5-flash...


## role="model" #

```python
calculate(expression='0.15 * 847')
```
#### Usage Information
  - [Token usage: 292/1048576]
  - [Agent budget usage: $0.0000/$10.00]
  - [Global budget usage: $0.0000/$200.00]
  - [Step 1/100]


## role="user" #
127.05
Asking gemini-2.5-flash...


## role="model" #

```python
finish(result='15% of 847 is 127.05')
```
#### Usage Information
  - [Token usage: 716/1048576]
  - [Agent budget usage: $0.0001/$10.00]
  - [Global budget usage: $0.0001/$200.00]
  - [Step 2/100]


## role="user" #
15% of 847 is 127.05


**Result:** 15% of 847 is 127.05

## 3. Custom Structured Output

Unlike other agentic systems, you do not need to specify the output schema for the agent. Just create a suitable "finish" function with parameters. The parameters could be treated as the top level keys in a json format.

In [3]:
import yaml
from IPython.display import display, Markdown
from kiss.core.kiss_agent import KISSAgent

def finish(success: bool, reasoning: str, output: str) -> str:
    """Finishes the current agent execution with success or failure, reasoning, and output.

    Args:
        success: True if the task was successful, False otherwise.
        reasoning: the reason you got the answer
        output: the final output as a number

    Returns:
        str: A YAML encoded dictionary containing 'success', 'reasoning', and 'output' keys.
    """
    result_str = yaml.dump(
        {
            "success": success,
            "reasoning": reasoning,
            "output": output,
        },
        indent=2,
        sort_keys=False,
    )
    return result_str
    
def calculate(expression: str) -> str:
    """Evaluate a math expression."""
    return str(eval(expression))

agent = KISSAgent(name="Math Buddy")
result = agent.run(
    model_name="gemini-2.5-flash",
    prompt_template="Calculate: {question}",
    arguments={"question": "What is 15% of 847?"},
    tools=[calculate, finish]
)
display(Markdown(f"""
### Structured Output Result

```yaml
{result}
```
"""))


## role="user" #
Calculate: What is 15% of 847?
Asking gemini-2.5-flash...


## role="model" #

```python
calculate(expression='0.15 * 847')
```
#### Usage Information
  - [Token usage: 335/1048576]
  - [Agent budget usage: $0.0000/$10.00]
  - [Global budget usage: $0.0001/$200.00]
  - [Step 1/100]


## role="user" #
127.05
Asking gemini-2.5-flash...


## role="model" #

```python
finish(output='127.05', success=True, reasoning='I calculated 15% of 847 using the calculate tool, which translated to 0.15 * 847.')
```
#### Usage Information
  - [Token usage: 829/1048576]
  - [Agent budget usage: $0.0001/$10.00]
  - [Global budget usage: $0.0001/$200.00]
  - [Step 2/100]


## role="user" #
"success": true
"reasoning": |-
  I calculated 15% of 847 using the calculate tool, which translated to 0.15 * 847.
"output": |-
  127.05




### Structured Output Result

```yaml
"success": true
"reasoning": |-
  I calculated 15% of 847 using the calculate tool, which translated to 0.15 * 847.
"output": |-
  127.05

```


## 4. Relentless Coding Agent - Multi-Agent Coding System

The Relentless Coding Agent is a multi-agent system with orchestration and sub-agents that relentlessly works to complete complex coding tasks by breaking them down into manageable sub-tasks.

In [7]:
from IPython.display import display, Markdown
from kiss.agents.coding_agents.relentless_coding_agent import RelentlessCodingAgent

# Create agent with a name
agent = RelentlessCodingAgent(name="My Coding Agent")

display(Markdown("""
## RelentlessCodingAgent Created Successfully!

**Key Features:**
- **Multi-Agent Architecture**: Orchestrator delegates to executor agents
- **Relentless Execution**: Auto-retries on failures with context
- **Path Access Control**: Enforces read/write permissions
- **Docker Support**: Optional container execution

---
### Running Example (requires API keys)
"""))

result = agent.run(
    prompt_template="Write, test, and optimize a fibonacci function",
    orchestrator_model_name="claude-sonnet-4-5",
    subtasker_model_name="claude-opus-4-5",
    readable_paths=["src/"],
    writable_paths=["output/"],
    base_dir=".",
    max_steps=100,
    trials=3
)
display(Markdown(f"""
---
### Result

{result}
"""))


KISSCodingAgent created successfully!

Key Features:
  - Multi-Agent Architecture: Orchestrator delegates to executor agents
  - Prompt Refinement: Auto-refines prompts on failures
  - Path Access Control: Enforces read/write permissions
  - Docker Support: Optional container execution

Example usage (requires API keys):
Executing task: Write, test, and optimize a fibonacci function

## role="user" #

## Task

Write, test, and optimize a fibonacci function

Call perform_subtask() to perform a sub-task.
perform_subtask() will return a yaml encoded dictionary containing the keys
'success' (boolean) and 'summary' (string).

Asking claude-sonnet-4-5...

Executing subtask: Write fibonacci function

## role="user" #

# Role

You are a software engineer who is expert with
bash commands and coding.

# Sub-task

Name: Write fibonacci function
Description: Write a fibonacci function that calculates the nth fibonacci number. Implement multiple versions including: 1) A simple recursive version, 2)

KeyboardInterrupt: 

## 5. Agent Evolver - Multi-Objective Agent Optimization (e.g. optimizing speed and cost)

The AgentEvolver creates and evolves AI agents using a Pareto frontier approach, optimizing for multiple objectives (e.g., token efficiency and execution time) through mutation and crossover operations.

In [ ]:
from IPython.display import display, Markdown
from kiss.agents.create_and_optimize_agent import AgentEvolver

# Create an AgentEvolver instance
evolver = AgentEvolver()

display(Markdown("## AgentEvolver Created Successfully!"))
display(Markdown("""
**Key Features:**
- **Pareto Frontier Maintenance**: Optimizes for multiple objectives (success, tokens, time)
- **Mutation**: Improve a single variant with targeted changes
- **Crossover**: Combine ideas from two variants
- **Crowding Distance Pruning**: Maintains diversity in the frontier
- **Lineage Tracking**: Records parent relationships across generations
"""))

# Define a task description for the agent to solve
task_description = """
Create a simple calculator agent that can:
1. Parse mathematical expressions from natural language
2. Perform basic arithmetic operations (+, -, *, /)
3. Return the result with an explanation

The agent must be efficient in token usage while maintaining accuracy.
"""

display(Markdown("---\n### Running Evolution (requires API keys)"))

# Run evolution to create and optimize agents for the task
best_variant = evolver.evolve(
    task_description=task_description,
    max_generations=2,        # small number of evolutionary generations for testing
    initial_frontier_size=1,  # Initial population size
    max_frontier_size=2,      # Maximum Pareto frontier size
    mutation_probability=0.8, # 80% mutation, 20% crossover
)

# Display the best variant's information
display(Markdown(f"""
---
### Best Variant Results

| Property | Value |
|----------|-------|
| **Path** | `{best_variant.folder_path}` |
| **Generation** | {best_variant.generation} |
| **Metrics** | {best_variant.metrics} |
"""))

# Get all variants on the Pareto frontier
frontier = evolver.get_pareto_frontier()
frontier_rows = "\n".join([
    f"| variant_{v.id} | {v.metrics.get('tokens_used', 0)} | {v.metrics.get('execution_time', 0):.2f}s |"
    for v in frontier
])
display(Markdown(f"""
### Pareto Frontier

| Variant | Tokens Used | Execution Time |
|---------|-------------|----------------|
{frontier_rows}
"""))

# Save the evolver state for later analysis
evolver.save_state("evolver_state.json")
display(Markdown("*State saved to `evolver_state.json`*"))

## AgentEvolver Created Successfully!


**Key Features:**
- **Pareto Frontier Maintenance**: Optimizes for multiple objectives (success, tokens, time)
- **Mutation**: Improve a single variant with targeted changes
- **Crossover**: Combine ideas from two variants
- **Crowding Distance Pruning**: Maintains diversity in the frontier
- **Lineage Tracking**: Records parent relationships across generations


---
### Running Evolution (requires API keys)

Starting AgentEvolver with 2 generations
Max frontier size: 2, Task: 
Create a simple calculator agent that can:
1. Parse mathematical expressions from natural language
2. Perform basic arithmetic operations (+, -, *, /)
3. Return the result with an explanation

The agent must be efficient in token usage while maintaining accuracy.


Initializing variant_1 agent
Running improvement on /var/folders/r2/wr1pwrgn36q3gg6248xtxf380000gn/T/tmpf0e7qu86/variant_1
Executing task: 
You have to optimize an AI agent for long-running complex tasks.

## Agent Requirements

  - The agent must be designed for **long-running, complex tasks** using
    the Agent API available at /Users/ksen/Dropbox/work/kiss/src.  Specifically, you should
    look at /Users/ksen/Dropbox/work/kiss/src/API.md and /Users/ksen/Dropbox/work/kiss/src/README.md first, and
    then look at code under the src folder as required.
    /Users/ksen/Dropbox/work/kiss/src/src/kiss/core/models/model_info.py contains information
    abou

## 9. Versioning

The project uses semantic versioning (MAJOR.MINOR.PATCH). The version is defined in `src/kiss/_version.py`.

In [18]:
from IPython.display import display, Markdown
from kiss import __version__

display(Markdown(f"**KISS Version:** `{__version__}`"))

KISS version: 0.1.7


## Summary

This notebook demonstrated the key features of the KISS Agent Framework:

1. **Simple Agent Creation**: Create agents with just a few lines of code
2. **Custom Output Formatting**: Define finish functions for structured output
3. **Multi-Agent Orchestration**: Compose agents using regular Python
4. **Self-Improving Agents**: Use prompt refinement for self-improvement
5. **Agent Evolution**: Optimize agents with Pareto frontier maintenance
6. **Coding Agents**: KISS, Claude, Gemini, and OpenAI coding agents
7. **Security Analysis**: ARVO vulnerability detection
8. **RAG Support**: SimpleRAG for retrieval-augmented generation
9. **Utility Agents**: Pre-built agents for common tasks

### How to Run This Notebook

```bash
# From the kiss directory, run:
uv run jupyter notebook kiss.ipynb

# Or use JupyterLab:
uv run jupyter lab kiss.ipynb
```

### Running as a Script

You can also convert this notebook to a Python script and run it:

```bash
uv run jupyter nbconvert --to script kiss.ipynb
uv run python kiss.py
```